In [1]:
import numpy as np 
import pandas as pd 
import math

In [2]:
df_train=pd.read_csv('salary-train.csv')
df_test=pd.read_csv('salary-test-mini.csv')

In [3]:
df_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [4]:
df_train.shape

(60000, 4)

In [5]:
X=df_train['FullDescription']

In [6]:
%%time
X=X.str.lower()

CPU times: user 156 ms, sys: 12 ms, total: 168 ms
Wall time: 167 ms


In [7]:
X.head()

0    international sales manager london ****k  ****...
1    an ideal opportunity for an individual that ha...
2    online content and brand manager// luxury reta...
3    a great local marketleader is seeking a perman...
4    registered nurse / rgn  nursing home for young...
Name: FullDescription, dtype: object

In [8]:
X = X.replace('[^a-zA-Z0-9]', ' ', regex = True)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer( min_df=5)
X_text=tfidf.fit_transform(X)

In [10]:
X_text.shape

(60000, 22861)

In [11]:
df_train[['LocationNormalized', 'ContractTime']].isnull().sum()

LocationNormalized        0
ContractTime          15582
dtype: int64

In [12]:
#df_train['LocationNormalized'].fillna('nan', inplace=True)
df_train['ContractTime'].fillna('nan', inplace=True)

In [13]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(df_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(df_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [14]:
from scipy import sparse
X_train=sparse.hstack([X_text,X_train_categ])

In [15]:
X_train.shape

(60000, 24627)

In [16]:
y=df_train['SalaryNormalized']

In [17]:
from sklearn import linear_model
regressor=linear_model.Ridge(alpha=1, random_state=241)
regressor.fit(X_train,y)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [18]:
X_test=df_test['FullDescription'].str.lower()

In [19]:
X_test = X_test.replace('[^a-zA-Z0-9]', ' ', regex = True)

In [20]:
X_test_text=tfidf.transform(X_test)

In [21]:
X_test=sparse.hstack([X_test_text,X_test_categ])

In [22]:
X_test.shape

(2, 24627)

In [23]:
z=regressor.predict(X_test)

In [24]:
print(round(z[0],2))
print(round(z[1],2))

56555.62
37188.32
